In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Colab Notebooks/ProgettoDRV

Mounted at /gdrive
/gdrive/My Drive/Colab Notebooks/ProgettoDRV


In [3]:
import pandas as pd
import numpy as np

# BITCOIN
bit = pd.read_csv('data/crypto/bitcoin.csv')
bit['Date'] = pd.to_datetime(bit["Date"])
bit = bit.set_index(['Date'])
bit = bit.asfreq("D", method="ffill")
bit = bit[bit.index >= pd.Timestamp("2018-01-01")]
bit = bit[bit.index <= pd.Timestamp("2022-03-10")]

# S&P500
sep = pd.read_csv("data/sep500.csv")
sep = sep.rename(columns={"Data":"Date"})
sep = sep.rename(columns={"Chiusura/Ultimo":"Close"})
sep.Date = pd.to_datetime(sep.Date)
sep = sep.set_index("Date")
sep = sep.asfreq("D", method="ffill")
sep = sep[sep.index >= pd.Timestamp("2018-01-01")]
sep = sep[sep.index <= pd.Timestamp("2022-03-10")]

# GOLD
goldETC = pd.read_csv("data/goldETC.csv")
goldETC.Date = pd.to_datetime(goldETC.Date)
goldETC = goldETC.set_index("Date")
goldETC = goldETC.asfreq("D", method="ffill")
goldETC = goldETC[goldETC.index >= pd.Timestamp("2018-01-01")]
goldETC = goldETC[goldETC.index <= pd.Timestamp("2022-03-10")]

In [ ]:
window = 252
# Bitcoin Drawdown
peak = bit.Close.max()
bit["Drawdown"] = (bit["Close"] - peak )/peak
Roll_Max = bit.Close.rolling(window, min_periods=1).max()
bit["Drawdown"] = (bit.Close/Roll_Max - 1.0)*100
bit["DrawdownMax"] = bit["Drawdown"].rolling(window, min_periods=1).min()

# S&P500 Drawdown
peak = sep.Close.max()
sep["Drawdown"] = (sep["Close"] - peak )/peak
Roll_Max = sep.Close.rolling(window, min_periods=1).max()
sep["Drawdown"] = (sep.Close/Roll_Max - 1.0)*100
sep["DrawdownMax"] = sep["Drawdown"].rolling(window, min_periods=1).min()

# GOLD ETC
peak = goldETC.Close.max()
goldETC["Drawdown"] = (goldETC["Close"] - peak )/peak
Roll_Max = goldETC.Close.rolling(window, min_periods=1).max()
goldETC["Drawdown"] = (goldETC.Close/Roll_Max - 1.0)*100
goldETC["DrawdownMax"] = goldETC["Drawdown"].rolling(window, min_periods=1).min()

In [ ]:
import plotly.graph_objects as go

import pandas as pd



# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(bit.index), y=list(bit.Drawdown), name = "Bitcoin", marker=dict(color="#FFA15A")))

fig.add_trace(
    go.Scatter(x=list(sep.index), y=list(sep.Drawdown), name = "S&P500", marker=dict(color="green")))

fig.add_trace(
    go.Scatter(x=list(goldETC.index), y=list(goldETC.Drawdown), name = "Gold", marker=dict(color="gold")))





fig.update_layout(
    # Modifica titolo
    title={
        'text': "Drawdown comparison",
        'y':0.90,
        'x':0.5,
        "font.size": 25,
        'xanchor': 'center',
        'yanchor': 'top'},
    # Imposta titoli asse x e y  
    #xaxis_title="Time",
    yaxis_title="Drawdown percentage"
)

fig.update_traces(
    #Modifica spessore linee
    line = dict(width = 2.5)
)

# Crea un'unica linea verticale per visualizzare tutti i valori
fig.update_layout(hovermode='x unified')

# Approssima a due cifre e mette il % dopo il numero
fig.update_traces(hovertemplate = '%{y:.2}%')

fig.show()


fig.write_html("./grafici/drawdown.html")

In [4]:
from pandas._libs.lib import to_object_array_tuples

# Compute the percentage of increment/decrement between the initial and final value
# Function used by montly_variation 
def pct_interest(initial_value, end_value):
  increase = end_value - initial_value
  return (increase/initial_value)*100

# Compute the increment/decrement of an investment for a month
# Month deve assumere valori numerici [1,12]
# Return the percentage/100 of variation of value of a stock for a month
# Function used by pac_computation
def montly_variation(stock_var, month, year):
  stock = stock_var
  if month == 12:
    init = stock.Close.loc[str(year)+"-"+str(month)+"-01"]
    final = stock.Close.loc[str(year+1)+"-01-01"]
  else:
    # Conditions required for a correct codification of the zero before numbers 1-9 
    if month<9:
      next_month = month+1
      next_month = "0"+str(next_month)
      month = "0"+str(month)
    if month == 9:
        next_month = 10
        month = "0"+str(month)
    else:
      next_month = int(month)+1   
    init = stock.Close.loc[str(year)+"-"+str(month)+"-01"]
    final = stock.Close.loc[str(year)+"-"+str(next_month)+"-01"]

  return (pct_interest(init, final))/100

# Compute the PAC (Piano Accumulo Capitale) for a given stock
# initial_invest: the quantity bought at the start of the investment
# montly_invest: quantity bought each first of the month
def pac_computation(stock_var, initial_invest, montly_invest, starting_year, ending_year):
  current_amount = initial_invest
  pac = pd.DataFrame(["Date", "Value"])
  # Flag for the first iteration of the cycle == first investment
  first_inv = True
  for year in range(starting_year, ending_year+1):
    
    for month in range(1,13):
      if first_inv:
        tmp = {"Date": str(str(year)+"-01-01"), "Value": current_amount}
        pac = pac.append(tmp, ignore_index = to_object_array_tuples)
        current_amount = current_amount + current_amount*montly_variation(stock_var, month, year)
        first_inv = False
      else:
        # if-else needed for the correct formatting of the date
        if month<10:
          tmp = {"Date": str(str(year)+"-0"+str(month)+"-01"), "Value": current_amount}
        else:
          tmp = {"Date": str(str(year)+"-"+str(month)+"-01"), "Value": current_amount}
        pac = pac.append(tmp, ignore_index = to_object_array_tuples)
        current_amount += montly_invest
        current_amount = current_amount + current_amount*montly_variation(stock_var, month, year)
  return pac
  

In [5]:
# COMPUTE PAC FOR BITCOIN, S&P500 and GOLD ETC
# FROM 2018 TO 2021
# WITH AN INITIAL INVESTMENT OF 1000$ AND
# AN INVESTMENT OF 200$ EACH MONTH

bit_pac = pac_computation(bit, 1000, 200, 2018, 2021)
# Coded needed for a correct dataframe:
# Drop first column
bit_pac = bit_pac.iloc[: , 1:]
# Drop first 2 rows
bit_pac = bit_pac.iloc[2: , :]
bit_pac['Date'] = pd.to_datetime(bit_pac["Date"])
bit_pac = bit_pac.set_index(['Date'])

sep_pac = pac_computation(sep, 1000, 200, 2018, 2021)
# Drop first column
sep_pac = sep_pac.iloc[: , 1:]
# Drop first 2 rows
sep_pac = sep_pac.iloc[2: , :]
sep_pac['Date'] = pd.to_datetime(sep_pac["Date"])
sep_pac = sep_pac.set_index(['Date'])

gold_pac = pac_computation(goldETC, 1000, 200, 2018, 2021)
# Drop first column
gold_pac = gold_pac.iloc[: , 1:]
# Drop first 2 rows
gold_pac = gold_pac.iloc[2: , :]
gold_pac['Date'] = pd.to_datetime(gold_pac["Date"])
gold_pac = gold_pac.set_index(['Date'])

In [6]:
import plotly.graph_objects as go

import pandas as pd



# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(bit_pac.index), y=list(bit_pac.Value), name = "Bitcoin", marker=dict(color="#FFA15A")))


fig.add_trace(
    go.Scatter(x=list(gold_pac.index), y=list(gold_pac.Value), name = "Gold", marker=dict(color="gold")))

fig.add_trace(
    go.Scatter(x=list(sep_pac.index), y=list(sep_pac.Value), name = "S&P500", marker=dict(color="green")))

fig.add_trace(go.Scatter(x=["2018-01-01","2021-12-01"], y=[1000,10600], name="Total investment",
                         line=dict(color='firebrick', width=1, dash='dash')))

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    # Modifica titolo
    title={
        'text': "PAC comparison: Bitcoin, Gold and S&P500",
        'y':0.90,
        'x':0.5,
        "font.size": 25,
        'xanchor': 'center',
        'yanchor': 'top'}
)
fig.update_traces(
    line = dict(width = 3)
)

# Crea un'unica linea verticale per visualizzare tutti i valori
fig.update_layout(hovermode='x unified')

# Approssima a tre cifre 
fig.update_traces(hovertemplate = '%{y:.3s}')


covid_annotation = [dict(x="2020-01-22",
                         y=7500,
                         showarrow=True,
                         arrowhead=2,
                         bordercolor="royalblue",
                         text="Beginning of the pandemic",
                         borderwidth=2,
                         borderpad=4,
                         bgcolor="lightblue",
                         opacity=0.8
                         )]

fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="right",
            active=0,
            x=0.2,
            y=1.1,
            buttons=list([
                dict(label="None",
                     method="update",
                     args=[{"visible": [True, True, True]},
                           {"annotations": []}]),
                dict(label="Covid-19",
                     method="update",
                     args=[{"visible": [True, True, True]},
                           {"annotations": covid_annotation}])
                ]))])

fig.show()

fig.write_html("./grafici/pac2018.html")

In [ ]:
bit_pac = pac_computation(bit, 1000, 200, 2020, 2021)
# Coded needed for a correct dataframe:
# Drop first column
bit_pac = bit_pac.iloc[: , 1:]
# Drop first 2 rows
bit_pac = bit_pac.iloc[2: , :]
bit_pac['Date'] = pd.to_datetime(bit_pac["Date"])
bit_pac = bit_pac.set_index(['Date'])

sep_pac = pac_computation(sep, 1000, 200, 2020, 2021)
# Drop first column
sep_pac = sep_pac.iloc[: , 1:]
# Drop first 2 rows
sep_pac = sep_pac.iloc[2: , :]
sep_pac['Date'] = pd.to_datetime(sep_pac["Date"])
sep_pac = sep_pac.set_index(['Date'])

gold_pac = pac_computation(goldETC, 1000, 200, 2020, 2021)
# Drop first column
gold_pac = gold_pac.iloc[: , 1:]
# Drop first 2 rows
gold_pac = gold_pac.iloc[2: , :]
gold_pac['Date'] = pd.to_datetime(gold_pac["Date"])
gold_pac = gold_pac.set_index(['Date'])

In [ ]:
import plotly.graph_objects as go

import pandas as pd



# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(bit_pac.index), y=list(bit_pac.Value), name = "Bitcoin", marker=dict(color="#FFA15A")))


fig.add_trace(
    go.Scatter(x=list(gold_pac.index), y=list(gold_pac.Value), name = "Gold", marker=dict(color="yellow")))

fig.add_trace(
    go.Scatter(x=list(sep_pac.index), y=list(sep_pac.Value), name = "S&P500", marker=dict(color="green")))


fig.add_trace(go.Scatter(x=["2020-01-01","2021-12-01"], y=[1000, 5800], name="Total investment",
                         line=dict(color='firebrick', width=1, dash='dash')))


fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    # Modifica titolo
    title={
        'text': "PAC comparison: Bitcoin, Gold and S&P500",
        'y':0.90,
        'x':0.5,
        "font.size": 25,
        'xanchor': 'center',
        'yanchor': 'top'}
)
fig.update_traces(
    line = dict(width = 3)
)

# Crea un'unica linea verticale per visualizzare tutti i valori
fig.update_layout(hovermode='x unified')


fig.show()

fig.write_html("./grafici/pac2020.html")

In [ ]:
bit_pac = pac_computation(bit, 1000, 200, 2021, 2021)
# Coded needed for a correct dataframe:
# Drop first column
bit_pac = bit_pac.iloc[: , 1:]
# Drop first 2 rows
bit_pac = bit_pac.iloc[2: , :]
bit_pac['Date'] = pd.to_datetime(bit_pac["Date"])
bit_pac = bit_pac.set_index(['Date'])

sep_pac = pac_computation(sep, 1000, 200, 2021, 2021)
# Drop first column
sep_pac = sep_pac.iloc[: , 1:]
# Drop first 2 rows
sep_pac = sep_pac.iloc[2: , :]
sep_pac['Date'] = pd.to_datetime(sep_pac["Date"])
sep_pac = sep_pac.set_index(['Date'])

gold_pac = pac_computation(goldETC, 1000, 200, 2021, 2021)
# Drop first column
gold_pac = gold_pac.iloc[: , 1:]
# Drop first 2 rows
gold_pac = gold_pac.iloc[2: , :]
gold_pac['Date'] = pd.to_datetime(gold_pac["Date"])
gold_pac = gold_pac.set_index(['Date'])

In [ ]:
import plotly.graph_objects as go

import pandas as pd



# Create figure
fig = go.Figure()

fig.add_trace(
    go.Scatter(x=list(bit_pac.index), y=list(bit_pac.Value), name = "Bitcoin", marker=dict(color="#FFA15A")))


fig.add_trace(
    go.Scatter(x=list(gold_pac.index), y=list(gold_pac.Value), name = "Gold", marker=dict(color="yellow")))

fig.add_trace(
    go.Scatter(x=list(sep_pac.index), y=list(sep_pac.Value), name = "S&P500", marker=dict(color="green")))


fig.add_trace(go.Scatter(x=["2021-01-01","2021-12-01"], y=[1000,3400], name="Total investment",
                         line=dict(color='firebrick', width=1, dash='dash')))

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.update_layout(
    # Modifica titolo
    title={
        'text': "PAC comparison: Bitcoin, Gold and S&P500",
        'y':0.90,
        'x':0.5,
        "font.size": 25,
        'xanchor': 'center',
        'yanchor': 'top'}
)
fig.update_traces(
    line = dict(width = 3)
)

fig.show()

fig.write_html("./grafici/pac2021.html")